In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import math
import random

NUM_POSITIONS = 16

MIN_LIST_LEN = 2
MAX_LIST_LEN = 2
MAX_STEPS = 10

SUCCESS_REWARD = 0.5
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.99
SHORTTERM_GAMMA = 0.7

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
LR_SCHEDULER_GAMMA = 0.93
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = '/home/mcwave/code/autocode/datasets/rl_sort_transformer_curriculum/list2_transformer4_192_gamma07_step10_v1'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    'less': 2,
    'equal': 3,
    'more': 4,
    '0': 5,
    '1': 6,
    '2': 7,
    '3': 8,
    '4': 9,
    '5': 10,
    '6': 11,
    '7': 12,
    '8': 13,
    '9': 14,
    '10': 15,
    '11': 16,
    '12': 17,
    '13': 18,
    '14': 19,
    '15': 20,
    'len1': 21,
    'len2': 22,
    'len3': 23,
    'len4': 24,
    'len5': 25,
    'len6': 26,
    'len7': 27,
    'len8': 28,
    'len9': 29,
    'len10': 30,
    'len11': 31,
    'len12': 32,
    'len13': 33,
    'len14': 34,
    'len15': 35,
    'len16': 36,
    'start0': 37,
    'start1': 38,
    'start2': 39,
    'start3': 40,
    'start4': 41,
    'start5': 42,
    'start6': 43,
    'start7': 44,
    'start8': 45,
    'start9': 46,
    'start10': 47,
    'start11': 48,
    'start12': 49,
    'start13': 50,
    'start14': 51,
    'start15': 52,
}
inv_vocab = {v: k for k, v in vocab.items()}

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

_, int_entropy = compute_entropy(4)

def get_entropy_of_integer(x):
    x = min(15, abs(x))
    return int_entropy[x]

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    entropy = sum([get_entropy_of_integer(x) for x in min_delta])
    if len(comparisons) == 1:
        return 3 * entropy
    else:
        return entropy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.start_pos = random.randint(0, NUM_POSITIONS // self.length - 1) * self.length
        self.list = [-1] * self.start_pos + [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [-1] * self.start_pos + [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        length_token = 'len{}'.format(self.length)
        start_pos_token = 'start{}'.format(self.start_pos)
        return vocab[length_token], vocab[start_pos_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_length(self):
        return self.length
    
    def get_start_pos(self):
        return self.start_pos

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                print("Error! Comparison without 2 indices!")
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 < self.start_pos or index1 >= self.start_pos + self.length \
                or index2 < self.start_pos or index2 >= self.start_pos + self.length:
                print(f"Error! Comparison with invalid indices {index1} {index2}")
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=192, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [3]:
def compute_bellman_returns(raw_rewards, gamma):
    bellman_returns = []
    R = 0
    for r in raw_rewards[::-1]:
        R = r + gamma * R
        bellman_returns.insert(0, R)
    return bellman_returns

# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=LR_SCHEDULER_GAMMA)
    
    # Optionally, load a checkpoint
    #checkpoint_path = os.path.join(OUTPUT_DIR, "ckpt_99000_0.9940_86.73.pth")
    #load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    total_steps = 0
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, start_pos, current_list = env.reset()
        input_tokens = [initial_token_id, start_pos]
        log_probs = []
        rewards = []
        comparisons = []
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                #print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.get_start_pos(), env.get_start_pos() + env.get_length())]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if done and reward == SUCCESS_REWARD:
                        success = True
                        if episode % 100 == 0:
                            print(decode(input_tokens, inv_vocab))
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list = env.step(action_tokens)
                if done and reward == SUCCESS_REWARD:
                    success = True
                    if episode % 100 == 0:
                        print(1, decode(input_tokens, inv_vocab))
                else:
                    pass
                    #reward += COMPARISON_ENTROPY_MULTIPLIER * compute_min_delta_entropy(comparisons)
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        success_rewards = [0.0] * len(rewards)
        if success: 
            num_successes += 1
            success_rewards[-1] = SUCCESS_REWARD

        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            success_rate = num_successes / episode_cnt
            avg_steps = total_steps / episode_cnt
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
            #
            # Reduce the lr
            scheduler.step()
            # Optionally, log the learning rate
            current_lr = scheduler.get_last_lr()[0]
            print(f"Learning rate = {current_lr:.6f}")
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns1 = compute_bellman_returns(rewards, SHORTTERM_GAMMA)
            returns2 = compute_bellman_returns(success_rewards, LONGTERM_GAMMA)
            returns = torch.tensor(np.array(returns1) + np.array(returns2)).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        total_steps += len(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{len(rewards)}, total reward:{sum(rewards):.4f}, {t2-t1} sec")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss:-3.6604, fail, steps:1, total reward:-10.0000, 0.33774447441101074 sec
Episode 1, loss:1.4148, succeed, steps:4, total reward:0.2000, 0.021348953247070312 sec
Episode 2, loss:1.1502, succeed, steps:4, total reward:0.2000, 0.018983840942382812 sec
Episode 3, loss:-0.1574, fail, steps:6, total reward:-0.6000, 0.027554035186767578 sec
Episode 4, loss:0.6391, succeed, steps:4, total reward:0.2000, 0.018658161163330078 sec
Episode 5, loss:0.6378, succeed, steps:4, total reward:0.2000, 0.018898725509643555 sec
Episode 6, loss:0.1922, succeed, steps:4, total reward:0.2000, 0.01862788200378418 sec
Episode 7, loss:0.2870, succeed, steps:4, total reward:0.2000, 0.018747806549072266 sec
Episode 8, loss:-10.7224, fail, steps:1, total reward:-10.0000, 0.005882978439331055 sec
Episode 9, loss:-0.3220, fail, steps:6, total reward:-0.6000, 0.02727484703063965 sec
Episode 10, loss:0.4920, succeed, steps:4, total reward:0.2000, 0.018557071685791016 sec
Episode 11, loss:0.2093, succeed, s

len2 start2 Comparison 2 3 more Swap
Episode 100, loss:0.0116, succeed, steps:4, total reward:0.2000, 0.018924236297607422 sec
Episode 101, loss:0.0227, succeed, steps:4, total reward:0.2000, 0.018703937530517578 sec
Episode 102, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.018369197845458984 sec
Episode 103, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.018482685089111328 sec
Episode 104, loss:0.0212, succeed, steps:4, total reward:0.2000, 0.018285274505615234 sec
Episode 105, loss:0.0355, succeed, steps:4, total reward:0.2000, 0.01868271827697754 sec
Episode 106, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01868581771850586 sec
Episode 107, loss:0.0113, succeed, steps:4, total reward:0.2000, 0.018530845642089844 sec
Episode 108, loss:0.0225, succeed, steps:4, total reward:0.2000, 0.0185244083404541 sec
Episode 109, loss:0.0120, succeed, steps:4, total reward:0.2000, 0.018523693084716797 sec
Episode 110, loss:0.0220, succeed, steps:4, total reward:0.2000, 0.

Episode 201, loss:0.0240, succeed, steps:4, total reward:0.2000, 0.01916337013244629 sec
Episode 202, loss:-40.0281, fail, steps:1, total reward:-10.0000, 0.006045341491699219 sec
Episode 203, loss:0.0086, succeed, steps:4, total reward:0.2000, 0.018459796905517578 sec
Episode 204, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.018680810928344727 sec
Episode 205, loss:0.0152, succeed, steps:4, total reward:0.2000, 0.018390655517578125 sec
Episode 206, loss:0.0151, succeed, steps:4, total reward:0.2000, 0.018282651901245117 sec
Episode 207, loss:0.0228, succeed, steps:4, total reward:0.2000, 0.018230676651000977 sec
Episode 208, loss:0.0084, succeed, steps:4, total reward:0.2000, 0.018895864486694336 sec
Episode 209, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.018573999404907227 sec
Episode 210, loss:0.0086, succeed, steps:4, total reward:0.2000, 0.018479585647583008 sec
Episode 211, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.018880605697631836 sec
Episode 21

Episode 296, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.0470578670501709 sec
Episode 297, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.04114842414855957 sec
Episode 298, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.039948463439941406 sec
Episode 299, loss:0.0118, succeed, steps:4, total reward:0.2000, 0.03486800193786621 sec
len2 start10 Comparison 10 11 more Swap
Episode 300, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.03475213050842285 sec
Episode 301, loss:0.0064, succeed, steps:4, total reward:0.2000, 0.032655954360961914 sec
Episode 302, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01562666893005371 sec
Episode 303, loss:0.0178, succeed, steps:4, total reward:0.2000, 0.01661849021911621 sec
Episode 304, loss:0.0063, succeed, steps:4, total reward:0.2000, 0.017453432083129883 sec
Episode 305, loss:0.0123, succeed, steps:4, total reward:0.2000, 0.01781177520751953 sec
Episode 306, loss:0.0119, succeed, steps:4, total reward:0.2000, 0.0

...

Episode 10737, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010955572128295898 sec
Episode 10738, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010734081268310547 sec
Episode 10739, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010626792907714844 sec
Episode 10740, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010568857192993164 sec
Episode 10741, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01060628890991211 sec
Episode 10742, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01065206527709961 sec
Episode 10743, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010608434677124023 sec
Episode 10744, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010606765747070312 sec
Episode 10745, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010571718215942383 sec
Episode 10746, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010550737380981445 sec
Episode 10747, loss:0.0001, succeed, steps:4, total reward:0.2000, 0.0105810165405

Episode 10830, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.011103153228759766 sec
Episode 10831, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010982036590576172 sec
Episode 10832, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01086878776550293 sec
Episode 10833, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010877847671508789 sec
Episode 10834, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010857105255126953 sec
Episode 10835, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010871648788452148 sec
Episode 10836, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010840892791748047 sec
Episode 10837, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01085519790649414 sec
Episode 10838, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01089930534362793 sec
Episode 10839, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01082754135131836 sec
Episode 10840, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010865211486816

Episode 10923, loss:0.0002, succeed, steps:4, total reward:0.2000, 0.011073112487792969 sec
Episode 10924, loss:0.0001, succeed, steps:4, total reward:0.2000, 0.010753631591796875 sec
Episode 10925, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01056218147277832 sec
Episode 10926, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010596513748168945 sec
Episode 10927, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010565996170043945 sec
Episode 10928, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010594367980957031 sec
Episode 10929, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010612010955810547 sec
Episode 10930, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010653495788574219 sec
Episode 10931, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010580778121948242 sec
Episode 10932, loss:0.0002, succeed, steps:4, total reward:0.2000, 0.010637998580932617 sec
Episode 10933, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010588169097

Episode 11015, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010918617248535156 sec
Episode 11016, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010772943496704102 sec
Episode 11017, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010620355606079102 sec
Episode 11018, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010645151138305664 sec
Episode 11019, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010621786117553711 sec
Episode 11020, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010584592819213867 sec
Episode 11021, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010654211044311523 sec
Episode 11022, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010641336441040039 sec
Episode 11023, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.01064300537109375 sec
Episode 11024, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010573148727416992 sec
Episode 11025, loss:0.0000, succeed, steps:4, total reward:0.2000, 0.010603189468


KeyboardInterrupt

